<a href="https://colab.research.google.com/github/emailic/YOLOv5-Microplasticos/blob/main/Conversion_json2txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


This code serves to convert LabelMe JSON files to YOLO format files. It works for both polygons as well as squares.
The code was executed on Google Colab (Jupyter Notebook).
The folders were located on Google drive.
LabelMe saves both Images and json files to one folder, in this case ../images/train. 
The code saves those
JSON files to '.../json_backup' folder, deletes them from the original '../images/train' folder
and finally adds the YOLO txt format files to the '../labels/train' folder.
Make sure to change the directory paths.

Code adapted from:  @ivder https://github.com/ivder

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:


import os
from os import walk, getcwd
from PIL import Image

def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)
    
    
"""-------------------------------------------------------------------""" 

""" Configure Paths"""   
mypath = '/content/drive/My Drive/microplasticos/images/train'
outpath = "/content/drive/My Drive/microplasticos/labels/train"
json_backup ="/content/drive/My Drive/microplasticos/json_backup"

wd = getcwd()
#list_file = open('%s_list.txt'%(wd), 'w')


'''correct .JPG to .jpg'''
#os.rename(old_name, new_name)


""" Get input json file list """
json_name_list = []
for file in os.listdir(mypath):
    if file.endswith(".json"):
        json_name_list.append(file)
    if '.JPG' in file:
        os.rename(mypath+'/'+file, mypath+'/'+file.strip('.JPG')+'.jpg')
    

In [ ]:


""" Process """
for json_name in json_name_list:
    txt_name = json_name.rstrip(".json") + ".txt"
    """ Open input text files """
    txt_path = mypath +'/' + json_name
    print("Input:" + txt_path)
    txt_file = open(txt_path, "r")
    
    """ Open output text files """
    txt_outpath = outpath + '/'+ txt_name
    print("Output:" + txt_outpath)
    txt_outfile = open(txt_outpath, "a+")

    """ Create a label dictionary."""
    label_dic={'fragment': 0, 'line': 1, 'organic': 2, 'pellet': 3, 'tar': 4}

    """ Convert the data to YOLO format """ 
    lines = txt_file.read().split() #'\r\n'  #for ubuntu, use "\r\n" instead of "\n"
    for idx, line in enumerate(lines):
        if ("lineColor" in line):
            break     #skip reading after find lineColor
        if ("label" in line):
            idxlist=[element for element in range(5,90,4)] #[5,9,13,17,21,25,29,33,37,41,45...]
            pl=list() #list of polygon points
            try:
                for i in idxlist:
                    pl.append((float(lines[idx+i].rstrip(',')), float(lines[idx+i+1])))
                    cls = label_dic[lines[idx+1].rstrip(',').strip('"')]
            except:
                pass
            #print('pl: ',pl)

            #in case when labelling, points are not in the right order
            xmin=9999999
            xmax=0
            ymin=9999999
            ymax=0
            for (x,y) in pl:
              if x<xmin:
                xmin=x
              if x>xmax:
                xmax=x
              if y<ymin:
                ymin=y
              if y>ymax:
                ymax=y
            img_path = str('%s/%s.jpg'%(mypath, os.path.splitext(json_name)[0]))

            im=Image.open(img_path)
      
            w= int(im.size[0])
            h= int(im.size[1])
        
            

            print(w, h)
            print(xmin, xmax, ymin, ymax)
            b = (xmin, xmax, ymin, ymax)
            bb = convert((w,h), b)
            print('CLASS', cls)
            print(bb)
            txt_outfile.write(str(cls) + " " + " ".join([str(a) for a in bb]) + '\n')
           
       
    os.rename(txt_path,json_backup+'/'+json_name)    #move json file to backup folder
  

Output streaming troncato alle ultime 5000 righe.
CLASS 1
(0.5315353881278538, 0.535697298325723, 0.03864630898021312, 0.03210616438356168)
4608 3456
2717.8082191780823 2817.8082191780823 1839.7260273972604 2146.5753424657537
CLASS 1
(0.6006528253424658, 0.5767218417047184, 0.021701388888888888, 0.08878741755454088)
4608 3456
2595.8904109589043 2682.1917808219177 1601.3698630136987 1679.4520547945206
CLASS 1
(0.5727085711567732, 0.4746559487569761, 0.018728595890410895, 0.02259322678843225)
4608 3456
2995.8904109589043 3089.041095890411 1691.7808219178082 1769.86301369863
CLASS 1
(0.6602573249619482, 0.5008165271435818, 0.02021499238964984, 0.02259322678843225)
4608 3456
3173.972602739726 3380.821917808219 1832.876712328767 2000.0
CLASS 1
(0.7112407248858447, 0.5545249873160831, 0.044889174277016726, 0.048357432775241)
4608 3456
3116.4383561643835 3300.0 2013.6986301369864 2135.6164383561645
CLASS 1
(0.6962281202435312, 0.6003059994926433, 0.03983542617960426, 0.03527714358193812)
4608